# Assignment 3
**by** 
- Yeoh Hui Qing
- Erik Rosvall


This is the coding part where we convert the csv files to RDF, based on the relations from our .owl file (from assignemnt2)

In [1]:
# importing libraries
import pandas as pd
import requests
from SPARQLWrapper import SPARQLWrapper, BASIC

To run this code, you might need the SPARQLWrapper lib.

Download for Windows:
```
pip install SPARQLWrapper
```

Download for Mac:
```
pip3 install SPARQLWrapper
```

**NOTE** It can take up to 25 min to run the code.

---

To read the CSV-files do we use the library pandas.

In [2]:
# reading the csv-files
courses = pd.read_csv('csv/Courses.csv')
assigned_hours = pd.read_csv('csv/Assigned_Hours.csv')
course_plannings = pd.read_csv('csv/Course_Plannings.csv')
programme_courses = pd.read_csv('csv/Programme_Courses.csv')
programmes = pd.read_csv('csv/Programmes.csv')
registrations = pd.read_csv('csv/Registrations.csv')
reported_hours = pd.read_csv('csv/Reported_Hours.csv')
senior_teachers = pd.read_csv('csv/Senior_Teachers.csv')
students = pd.read_csv('csv/Students.csv')
teaching_assistants = pd.read_csv('csv/Teaching_Assistants.csv')
course_instances = pd.read_csv('csv/Course_Instances.csv')

## Methods

In [3]:
def run_query(slist):
    """
    run_query takes in a list and adds extra syntax to creat an proper SPARQL-query.
    
    Params:
    -------
    slist: is a list of all rdf's from a csv file.
    """
    sparql = get_wrapper()
    prefixes = get_prefixes()
    
    ll = ''.join(slist)
    queryString = prefixes + "INSERT DATA {" + ll + "}"
    sparql.method = 'POST'
    sparql.setQuery(queryString)
    sparql.queryType = "INSERT"
    res = sparql.query()

# 
def get_prefixes() -> str:
    """
    This is the prefixes generated from protege, can be foun in the .owl-file
    
    Returns
    -------
    Returns a string 
    """
    prefixes = '''prefix : <http://www.semanticweb.org/huiqing/ontologies/2022/3/untitled-ontology-2#> 
            prefix owl: <http://www.w3.org/2002/07/owl#> 
            prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
            prefix xml: <http://www.w3.org/XML/1998/namespace> 
            prefix xsd: <http://www.w3.org/2001/XMLSchema#> 
            prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>                     
            '''
    return prefixes

def get_wrapper():
    """
    returns teh connection string needed to upload any relations to the database
    """
    sparql = SPARQLWrapper('http://Ezz:7200/repositories/assignment3/statements')
    return sparql

## Uploading data

Down below are the custom code to upload to the database

---

This is to time the total execution of loading the data to GraphDB.

In [1]:
import time
t0 = time.time()

### Setting parent classes

GraphDB can not set the parentclasses by default, this block below enforce the heirarchy before inserting the data to teh database

In [4]:
# Set subclasses
qstring = ""
subclasses = []

qstring += ":TeachingAssistant rdfs:subClassOf :TeachingStaff . "
qstring += ":SeniorTeacher rdfs:subClassOf :TeachingStaff . "
qstring += ":Students rdfs:subClassOf :Individual . "
qstring += ":TeachingStaff rdfs:subClassOf :Individual . "

subclasses += qstring
run_query(subclasses)

### Inserting data

In [5]:
# Set subclasses
qstring = ""
subclasses = []

qstring += ":TeachingAssistant rdfs:subClassOf :TeachingStaff . "
qstring += ":SeniorTeacher rdfs:subClassOf :TeachingStaff . "
qstring += ":Students rdfs:subClassOf :Individual . "
qstring += ":TeachingStaff rdfs:subClassOf :Individual . "

subclasses += qstring  
run_query(subclasses)

In [6]:
# Running for students
qstring = ""
slist = []

for index, row in students.iterrows():
    qstring = ""
    curr = row
    qstring += ":%s rdf:type :Students ; "%curr['Student id']
    qstring += ":hasId :%s ; "%curr['Student id']
    qstring += ":hasRegistrationYear :%d ; "%(curr['Year'])
    qstring += ":hasGraduated '%s' ; "%curr['Graduated']
    qstring += ":hasName '%s' ."%curr['Student name']
    slist += qstring
    if index == len(students) - 1 or index%10000 == 1:
        run_query(slist)
        slist = []

In [7]:
# Senior Teacher + Division/Department
qstring = ""
stlist = []

for index, row in senior_teachers.iterrows():
    qstring = ""
    curr = row
    qstring += ":%s rdf:type :Department . "%curr['Department name']
    qstring += ":%s rdf:type :Division ; "%curr['Division name']
    qstring += ":hasDivisionId :%s ; "%curr['Division name']
    qstring += ":isPartOf :%s . "%curr['Department name']
    
    qstring += ":%s rdf:type :SeniorTeacher ; "%curr['Teacher id']
    qstring += ":isWorkingFor :%s ; "%curr['Division name']
    qstring += ":hasId :%s ; "%curr['Teacher id']
    qstring += ":hasName '%s' ."%curr['Teacher name']
    stlist += qstring
    
run_query(stlist)
stlist = []

In [8]:
# Teaching Assistants
qstring = ""
talist = []

for index, row in teaching_assistants.iterrows():
    qstring = ""
    curr = row
    qstring += ":%s rdf:type :Department . "%curr['Department name']
    qstring += ":%s rdf:type :Division ; "%curr['Division name']
    qstring += ":isPartOf :%s . "%curr['Department name']
    
    qstring += ":%s rdf:type :TeachingAssistant ; "%curr['Teacher id']
    qstring += ":isWorkingFor :%s ; "%curr['Division name']
    qstring += ":hasId :%s ; "%curr['Teacher id']
    qstring += ":hasName '%s' ."%curr['Teacher name']
    talist += qstring
    
run_query(talist)
talist = []

In [9]:
# Programmes and Department
qstring = ""
plist = []

for index, row in programmes.iterrows():   
    qstring = ""
    curr = row
    qstring += ":%s rdf:type :Programme ; "%curr['Programme code']
    qstring += ":hasDirector :%s ; "%curr['Director']
    qstring += ":hasProgrammeCode :%s ; "%curr['Programme code']
    qstring += ":hasProgrammeName :%s ."%curr['Programme name']
    qstring += ":%s rdf:type :Department ; "%curr['Department name']
    qstring += ":isOffering :%s ."%curr['Programme code']
    plist += qstring
run_query(plist)

In [10]:
# Running queries to insert data to Programmes

qstring = ""
proglist = []

for index, row in students.iterrows():
    proglist += ":%s :isEnrolledIn :%s . "%(row['Student id'], row['Programme'])
    if index == len(students) - 1 or index%10000 == 1:
        run_query(proglist)
        proglist = []


In [11]:
# Running queries to insert data to Department
qstring = ""
stlist = []

for index, row in senior_teachers.iterrows():
    qstring = ""
    curr = row
    qstring += ":%s :isWorkingFor :%s . "%(curr['Teacher id'], curr['Department name'])
    stlist += qstring
    if index == len(senior_teachers) - 1 or index%10000 == 1:
        run_query(stlist)
        stlist = []
        
qstring = ""
talist = []

for index, row in teaching_assistants.iterrows():
    qstring = ""
    curr = row
    qstring += ":%s :isWorkingFor :%s . "%(curr['Teacher id'], curr['Department name'])
    talist += qstring
    if index == len(teaching_assistants) - 1 or index%10000 == 1:
        run_query(talist)
        talist = []

In [12]:
# Courses
qstring = ""
clist = []

for index, row in courses.iterrows():
    qstring = ""
    curr = row
    qstring += ":%s rdf:type :Courses ; "%curr['Course code']
    qstring += ":isOwnedBy :%s ; "%curr['Owned By']
    qstring += ":hasLevel '%s' ; "%curr['Level']
    qstring += ":hasCredits '%s' ; "%curr['Credits']
    qstring += ":hasCourseCode '%s' ; "%curr['Course code']
    qstring += ":hasCourseName '%s' ;"%curr['Course name']
    qstring += ":isUnder :%s ."%curr['Division']
    qstring += ":%s :isPartOf :%s . "%(curr['Division'], curr['Department'])
    clist += qstring
    if index == len(courses) - 1 or index%10000 == 1:
        run_query(clist)
        clist = []

In [13]:
# Programme Courses
qstring = ""
cplist = []

for index, row in programme_courses.iterrows():
    qstring = ""
    curr = row
    qstring += ":ProgrammeCourse%s rdf:type :ProgrammeCourses ; "%index
    qstring += ":isBelongingTo :%s ; "%(curr['Course'])
    qstring += ":hasStudyYear '%s' ; "%curr['Study Year']
    qstring += ":hasAcademicYear '%s' ; "%curr['Academic Year'] # Attached to CourseInstance instead
    qstring += ":hasCourseType '%s' ."%curr['Course Type']
    
    # necessary to differentiate ProgrammeCourse from Programme 
    qstring += ":%s :isProviding :ProgrammeCourse%s . "%(curr['Programme code'], index) 
    
    cplist += qstring
    if index == len(programme_courses) - 1 or index%5000 == 1:
        run_query(cplist)
        cplist = []

In [14]:
courseInstancePlanning = course_instances.merge(course_plannings, left_on='Instance_id', right_on='Course')
courseInstancePlanning.head()

,Course code,Study period,Academic year,Instance_id,Examiner,Course,Planned number of Students,Senior Hours,Assistant Hours
0,1763,1.0,2010-2011,I-1,19581118-0029,I-1,100,260,700
1,1556,2.0,2010-2011,I-2,19680307-0062,I-2,150,240,700
2,1516,4.0,2010-2011,I-3,19691216-0147,I-3,100,260,560
3,1293,3.0,2010-2011,I-4,19650728-0047,I-4,150,280,360
4,1741,3.0,2010-2011,I-5,19671119-0137,I-5,150,260,700


In [15]:
# Courses Instance + The course planning (we combinded them in assignment2)
qstring = ""
cilist = []

for index, row in courseInstancePlanning.iterrows():
    qstring = ""
    curr = row
    qstring += ":%s rdf:type :CourseInstances ; "%curr['Instance_id']
    qstring += ":hasAcademicYear '%s' ; "%curr['Academic year']
    qstring += ":hasStudyPeriod '%s' ; "%curr['Study period']
    qstring += ":hasPlanningNumStudents %s ; "%curr['Planned number of Students']
    qstring += ":needsSeniorHours %s ; "%curr['Senior Hours']
    qstring += ":needsAssistantHours %s . "%curr['Assistant Hours']
    qstring += ":%s :isExamining :%s . "%(curr['Examiner'], curr['Instance_id'])
    qstring += ":%s :isConductedAs :%s . "%(curr['Course code'], curr['Instance_id'])
    cilist += qstring
    if index == len(courseInstancePlanning) - 1 or index%2000 == 1:
        run_query(cilist)
        cilist = []

In [16]:
# Registration
qstring = ""
rlist = []

for index, row in registrations.iterrows():
    qstring = ""
    curr = row
    qstring += ":Registration%s rdf:type :Registration ; "%index
    qstring += ":hasStatus '%s' ; "%curr['Status']
    qstring += ":hasGrade '%s' ;"%curr['Grade']
    qstring += ":isRegisteredFor :%s ."%curr['Course Instance']
    qstring += ":%s :isRegisteredUnder :Registration%s ."%(curr['Student id'], index)
    rlist += qstring
    if index == len(registrations) - 1 or index%10000 == 1:
        run_query(rlist)
        rlist = []

In [17]:
merged_hours = assigned_hours.merge(reported_hours, left_on=['Course Instance', 'Teacher Id'], right_on=['Course code', 'Teacher Id'])
merged_hours.head()

,Course code_x,Study Period,Academic Year,Teacher Id,Hours_x,Course Instance,Course code_y,Hours_y
0,1763,1.0,2010-2011,19680307-0062,240.0,I-1,I-1,229.0
1,1763,1.0,2010-2011,19770310-0188,100.0,I-1,I-1,115.0
2,1763,1.0,2010-2011,19901025-0328,120.0,I-1,I-1,120.0
3,1763,1.0,2010-2011,19920428-0326,140.0,I-1,I-1,126.0
4,1556,2.0,2010-2011,19670913-0054,240.0,I-2,I-2,238.0


Since TimeAssignment is a weak entity dependent on the TeacherID, we can also join on it to uniquely identify the time assignment with its corresponding assignedHours and reportedHours.

In [18]:
# ReportedHours and AssignedHours
qstring = ""
mhlist = []

for index, row in merged_hours.iterrows():
    qstring = ""
    curr = row
    qstring += ":TimeAssignment%s rdf:type :TimeAssignment ; "%index #cannot use CourseCode to uniquely identify
    qstring += ":hasAssignedHours %s ; "%curr['Hours_x']
    qstring += ":hasReportedHours %s ; "%curr['Hours_y']
    qstring += ":isNeededFor :%s . "%curr['Course Instance']
    qstring += ":%s :isAssigned :TimeAssignment%s . "%(curr['Teacher Id'], index)
    mhlist += qstring
    if index == len(merged_hours) - 1 or index%10000 == 1:
        run_query(mhlist)
        mhlist = []

In [5]:
t1 = time.time()
total = t1-t0
print('Time (sec):',total)

Time (sec): 30.63567066192627


In [3]:
m = total/60
print(f'Total time in min: {m}')

Total time in min: 0.09568355480829875
